# RandomForestRegressor Con series Temporales
### (TimeSeriesSplit)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
%matplotlib inline

url ="https://raw.githubusercontent.com/albercol/TFG/main/Fase2/Fase%202.3/DatosUnificados/Cont_Meteo_Traf.csv"
df = pd.read_csv(url,parse_dates=['FECHAH'])
#df["HORA"] = df['FECHAH'].dt.hour ##Añadido
#df["DIA"] = df['FECHAH'].dt.day ##Añadido
#df["MES"] = df['FECHAH'].dt.month ##Añadido
df = df.dropna()

#Transformamos la columna tipo en 3 columnas numericas para cada tipo (LABORABLE, SABADO, DOMINGO, FESTIVO)
valores = df.TIPO.unique()
for v in valores:
    print(v)
    df[v] = df.TIPO.apply(lambda x: 1 if x==v else 0)

#Borramos las columnas que no son necesarias.
df = df.drop(['TIPO', 'NO', 'PM10', 'NOx'], axis=1)
df=df.set_index(['FECHAH'])
df.dropna(inplace=True)
df

LABORABLE
SABADO
DOMINGO
FESTIVO


,CO,NO2,PM2.5,O3,VELOCIDAD VIENTO,DIR. VIENTO,TEMPERATURA,HUMEDAD RELATIVA,PRESION BAROMETRICA,PRECIPITACIONES,ES10,ES53,ESSUMA,LABORABLE,SABADO,DOMINGO,FESTIVO
FECHAH,,,,,,,,,,,,,,,,,
2019-08-01 00:00:00,0.2,22.0,10.0,58.87,1.84,97.0,26.1,52.0,943.0,0.0,223.0,347.0,570.0,1,0,0,0
2019-08-01 01:00:00,0.2,15.0,10.0,63.73,1.97,117.0,24.9,55.0,943.0,0.0,173.0,231.0,404.0,1,0,0,0
2019-08-01 02:00:00,0.1,14.0,9.0,66.50,1.72,96.0,24.0,55.0,943.0,0.0,107.0,180.0,287.0,1,0,0,0
2019-08-01 03:00:00,0.1,14.0,10.0,66.62,1.55,106.0,23.3,55.0,943.0,0.0,80.0,129.0,209.0,1,0,0,0
2019-08-01 04:00:00,0.2,15.0,10.0,62.57,1.13,67.0,22.9,57.0,943.0,0.0,89.0,105.0,194.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-25 10:00:00,0.3,38.0,9.0,47.20,1.37,79.0,18.3,46.0,953.0,0.0,587.0,658.0,1245.0,1,0,0,0
2021-05-25 11:00:00,0.3,37.0,7.0,60.22,1.93,62.0,19.3,42.0,953.0,0.0,582.0,651.0,1233.0,1,0,0,0
2021-05-25 12:00:00,0.2,31.0,7.0,73.93,1.53,12.0,21.2,38.0,952.0,0.0,573.0,743.0,1316.0,1,0,0,0


In [3]:
def pintarGrafico(y_pred, y_test, nombre, mae):
    cuantos = 8
    ypred = y_pred[:cuantos]
    ylabel = list(y_test)[:cuantos]
    x = list(range(len(ylabel)))
    plt.rcParams["font.size"] = 10
    figure, ax = plt.subplots(1,figsize=(4, 4),dpi=100)    
    for i in range(len(x)):
        plt.plot([x[i],x[i]],[ypred[i],ylabel[i]],color="darkorange")

    plt.title("Prediccion para " + nombre + " mae: " + str(round(mae, 3)))
    plt.scatter(x,ypred, color='blue', label="pred",s=8)
    plt.scatter(x,ylabel, color='green', label="label",s=4)

    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.show()

In [31]:
def split_test(df,  predict, horas, hAnterior):
    #asignamoes una variable al modelo y seleccionamos el mejor mae.
    model_chos, mae_ctrl = None, 5000
    
    colsX = [ 'VELOCIDAD VIENTO', 'DIR. VIENTO', 'TEMPERATURA', 'HUMEDAD RELATIVA',
           'PRESION BAROMETRICA', 'PRECIPITACIONES', 'ES10','ES53','ESSUMA' ,"LABORABLE",
             "SABADO","DOMINGO", "FESTIVO"]
     
    #Se le añade el valor predicho del la hora anterior.
    if hAnterior:
        colsX.append("pred")
        #Añadimos la columna con la prediccion de la hora anterior
        df["pred"] = df[predict].shift(+1)
        df = df[1:]
    
    X = df[colsX].values
    y = df[predict].values
    
    test = []
    pred = []
    for i in range(horas):
        U=[] 
        Y=[]
        test.append(U)
        pred.append(Y)

    n_splits =15
    splits = TimeSeriesSplit(n_splits=n_splits, test_size=horas) #testsize=8 porque queremos predecir a 8 horas futuras.
    model = RandomForestRegressor()
    print(splits)
    
    for step, (train_index, test_index) in enumerate(splits.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if len(predict) == 1: 
            y_train = y_train.ravel()
            
        #entrenamos el modelo
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        for i in range(horas):
            test[i].append(y_test[i])
            pred[i].append(y_pred[i])
           
    return test, pred, df


In [ ]:
#Predecimos a 8 horas futuras.
horas = 8

predict = ['NO2']
df2 = df
test, pred, df2 = split_test(df2, predict, horas, False)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=15, test_size=8)


In [27]:
maes = []
for i in range(horas):
    maes.append(mean_absolute_error(test[i],pred[i]))


In [28]:
print(maes)

[10.5633, 10.27425, 10.1985, 10.374299999999998, 9.35845, 10.6356, 11.36255, 10.8073]


In [30]:
print(test)

[[array([38.]), array([16.]), array([23.]), array([38.]), array([36.]), array([21.]), array([51.]), array([42.]), array([32.]), array([120.]), array([40.]), array([24.]), array([125.]), array([53.]), array([24.]), array([148.]), array([49.]), array([34.]), array([79.]), array([65.]), array([13.]), array([71.]), array([18.]), array([25.]), array([66.]), array([36.]), array([23.]), array([32.]), array([67.]), array([29.]), array([43.]), array([77.]), array([33.]), array([48.]), array([65.]), array([35.]), array([36.]), array([32.]), array([15.]), array([38.]), array([18.]), array([12.]), array([58.]), array([32.]), array([25.]), array([81.]), array([23.]), array([21.]), array([81.]), array([65.]), array([28.]), array([83.]), array([71.]), array([13.]), array([42.]), array([49.]), array([34.]), array([18.]), array([40.]), array([32.]), array([28.]), array([47.]), array([39.]), array([57.]), array([31.]), array([20.]), array([69.]), array([14.]), array([36.]), array([17.]), array([55.]), a